In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

You may need to refer [this](http://stackoverflow.com/questions/35114376/error-when-computing-summaries-in-tensorflow)

In [2]:
# reading mnist using libraries shipped with tf
mnist = input_data.read_data_sets("../data/mnist/", one_hot=True) 

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 0.5
number_of_epochs = 1000
batch_size = 100
logs_path = "logs/"

In [4]:
#tf model building

with tf.name_scope('Model'):
    
    X = tf.placeholder(tf.float32, [None, 784])
    Y = tf.placeholder(tf.float32, [None, 10])
    W = tf.Variable(tf.random_normal([784, 10]))
    b = tf.Variable(tf.zeros([1, 10]))

with tf.name_scope('Prediction'):
    softmax_prediction = tf.nn.softmax(tf.add(tf.matmul(X, W), b))

with tf.name_scope('Loss'):
    cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(softmax_prediction), reduction_indices=[1]))

with tf.name_scope('Train'):
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy_loss)

with tf.name_scope('Accuracy'):
    count_correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(softmax_prediction,1))
    accuracy = tf.reduce_mean(tf.cast(count_correct_prediction, tf.float32))

init_op = tf.initialize_all_variables()

tf.scalar_summary("cross-entropy-loss", cross_entropy_loss)
tf.scalar_summary("accuracy", accuracy)

summary_op = tf.merge_all_summaries()

In [5]:
with tf.Session() as sess:
    sess.run(init_op)
    summary_writer = tf.train.SummaryWriter(logs_path, graph=tf.get_default_graph())
    for epoch in xrange(number_of_epochs):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, summary_data = sess.run([train_step, summary_op], feed_dict={X: batch_xs, Y: batch_ys})
        
        if(epoch%10==0):
            summary_writer.add_summary(summary_data, epoch)
    print "Training Accuracy = "+str(sess.run(accuracy, feed_dict={X: mnist.train.images, Y: mnist.train.labels}))
    print "Testing Accuracy = "+str(sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Training Accuracy = 0.877909
Testing Accuracy = 0.8828
